# Introductory tutorial for the synIR library and its python bindings
Welcome to the synIR (/ˈsɪnɚ/) library!

This jupyter notebook showcases the basic usage of our quantum compilation library.

## Library overview
This compilation library is a rust-based library with python bindings. 

synIR stands for SYNthesizable Intermediate Representation. Unlike gate-based intermediate representations (e.g. QIR), the library represents quantum circuits as sequences of exponentiated Paulistrings and a Clifford tableau $A$: 

$$U = A\Pi (\Pi_{P \in \{I, X, Y, Z\}^N} e^{-\alpha P})$$

Here, the inner product of $e^{-\alpha P}$ is assumed to be mutually commuting. This assumption helps optimize circuits resulting from a trotterization process in exchange for a potential increase of the Trotter Error.

### Lexicon
Since there are some conflicting terminology in literature, we will define what we mean by some of these terms here in alphabetical order.

**Pauli Exponential** is the universal representation that we use in the library, it contains a Clifford tableau and a sequence of Pauli Polynomials. An exponential is a product and since the product of matrices is not commutative, the sequences in the Pauli Exponential do not commute.

**Pauliletter** is the letter by which we refer to any of the four Pauli matrices: $I$, $X$, $Y$, and $Z$.

**Pauli Polynomial** is the subsequence of Paulistrings in the Pauli Exponential that we assume to be mutually commuting. An example of this is the Phase Polynomial, which is a Pauli Polynomial consisting only of I and Z Pauliletters. The naming scheme is analogous to the Phase Polynomial. Similarly, a polynomial contains a sum of terms and since the sum of matrices is commutative, the terms in the Pauli Polynomial should also commute.

**Paulistring** is a sequence of Pauli letters representing the tensor product of the respective Pauli matrices.


## Basic data structures

In [ ]:
:dep syn = { path = "../." } // If you run the syn package locally

In [ ]:
use syn::data_structures::{PauliString, PauliPolynomial, CliffordTableau}
use syn::ir::pauli_exponential::PauliExponential

In [3]:
let pauli_string = "IXYZ";
let paulivec = PauliString::from_text(pauli_string);

println!("{}", paulivec);

I X Y Z


In [4]:
let ham = vec![("IXYZ", 0.3), ("XXII", 0.7), ("YYII", 0.12)];
let pp = PauliPolynomial::from_hamiltonian(ham);
//println!("{}", pp); // WIP

println!("Pauli Polynomial size:{} qubits x {} terms", pp.size(), pp.length());

print!(" | ");
for i in 0..pp.length(){
    print!("{} ", pp.angle(i));
}
println!(" | ");
for i in 0..pp.size(){
    println!("QB{} | {} |", i, pp.chains()[i]);
}

Pauli Polynomial size:4 qubits x 3 terms
 | 0.3 0.7 0.12  | 
QB0 | I X Y |
QB1 | X X Y |
QB2 | Y I I |
QB3 | Z I I |


()

In [5]:
use syn::data_structures::PropagateClifford; // Would be nice if this was not needed when importing CliffordTableau

In [6]:
let mut ct = CliffordTableau::new(4);
println!("{}", ct);
ct.v(1);

println!("\nAfter an V gate:\n{}", ct);
ct.v(1);

println!("\nAfter an V gate:\n{}", ct);

CliffordTableau(4)
X I I I Z I I I
I X I I I Z I I
I I X I I I Z I
I I I X I I I Z
+ + + + + + + +

After an V gate:
CliffordTableau(4)
X I I I Z I I I
I X I I I Y I I
I I X I I I Z I
I I I X I I I Z
+ + + + + - + +

After an V gate:
CliffordTableau(4)
X I I I Z I I I
I X I I I Z I I
I I X I I I Z I
I I I X I I I Z
+ + + + + - + +


## A Toffoli gate example

In [7]:
let angles = [0.25, 0.25, 0.25, -0.25, -0.25, -0.25, 0.25];
let mut ccz_ham = vec!["ZII", "IZI", "IIZ", "ZZI", "ZIZ", "IZZ", "ZZZ"];
let mut ccz_pp = PauliPolynomial::from_hamiltonian(ccz_ham.iter().copied().zip(angles).collect());
let mut ccz_ct = CliffordTableau::new(3);
ccz_ct.h(2); // Add the final H around the CCZ
// Add the first H before the CCZ
ccz_pp.h(2);
ccz_ct.h(2);
let toffoli = PauliExponential::new(std::collections::VecDeque::from([ccz_pp]), ccz_ct); 
//toffoli.h(2); // This does not work yet, so we do this to the separate parts, so we did this earlier.
//toffoli // Does not work yet before visualization

### Define a circuit class because our rust code does not have one

In [8]:
use syn::ir::{CliffordGates, Gates};

In [9]:
type Angle = f64;
#[derive(Debug, Default)]
pub struct MockCircuit {
    commands: Vec<MockCommand>,
}

#[derive(Debug, PartialEq)]
pub enum MockCommand {
    CX(syn::IndexType, syn::IndexType),
    CZ(syn::IndexType, syn::IndexType),
    X(syn::IndexType),
    Y(syn::IndexType),
    Z(syn::IndexType),
    H(syn::IndexType),
    S(syn::IndexType),
    V(syn::IndexType),
    SDgr(syn::IndexType),
    VDgr(syn::IndexType),
    Rx(syn::IndexType, Angle),
    Ry(syn::IndexType, Angle),
    Rz(syn::IndexType, Angle),
}

impl MockCircuit {
    pub fn new() -> Self {
        Self {
            commands: Vec::new(),
        }
    }
    pub fn commands(&self) -> &Vec<MockCommand> {
        &self.commands
    }
}

impl CliffordGates for MockCircuit {
    fn s(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::S(target));
    }

    fn v(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::V(target));
    }

    fn s_dgr(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::SDgr(target));
    }

    fn v_dgr(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::VDgr(target));
    }

    fn x(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::X(target));
    }

    fn y(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::Y(target));
    }

    fn z(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::Z(target));
    }

    fn h(&mut self, target: syn::IndexType) {
        self.commands.push(MockCommand::H(target));
    }

    fn cx(&mut self, control: syn::IndexType, target: syn::IndexType) {
        self.commands.push(MockCommand::CX(control, target));
    }

    fn cz(&mut self, control: syn::IndexType, target: syn::IndexType) {
        self.commands.push(MockCommand::CZ(control, target));
    }
}

impl Gates for MockCircuit {
    fn rx(&mut self, target: syn::IndexType, angle: Angle) {
        self.commands.push(MockCommand::Rx(target, angle));
    }

    fn ry(&mut self, target: syn::IndexType, angle: Angle) {
        self.commands.push(MockCommand::Ry(target, angle));
    }

    fn rz(&mut self, target: syn::IndexType, angle: Angle) {
        self.commands.push(MockCommand::Rz(target, angle));
    }
}


### Synthesize the circuit

In [10]:
use syn::ir::pauli_exponential::PauliExponentialSynthesizer;
use syn::ir::clifford_tableau::CliffordTableauSynthStrategy;
use syn::ir::pauli_polynomial::PauliPolynomialSynthStrategy;
use syn::ir::Synthesizer;

In [11]:
let mut circuit = MockCircuit::new();
let mut synthesizer = PauliExponentialSynthesizer::from_strategy(
    PauliPolynomialSynthStrategy::Naive,
    CliffordTableauSynthStrategy::Naive,
);
synthesizer.synthesize(toffoli, &mut circuit);
circuit

MockCircuit { commands: [Rz(0, 0.25), Rz(1, 0.25), H(2), Rz(2, 0.25), CX(0, 1), Rz(1, -0.25), CX(0, 2), Rz(2, -0.25), CX(1, 2), Rz(2, -0.25), CX(0, 2), Rz(2, 0.25), CX(0, 1), CX(0, 2), CX(1, 2), H(2)] }

## FAQ

### How do I make a circuit?
You don't. 

The rust part of the library does not have a notion of quantum circuit and this is by design. The only data structure that contains what the quantum program does is our intermediate representation. That intermediate representation can consume gates and generate gates in a way that is defined by the `PropagateClifford`, `MaskedPropagateClifford`, `Synthesizer`, and `AdjointSynthesizer` traits.

Circuits are a concept that are practically needed for integration, so we translate circuit classes from external libraries directly to our intermediate representation without relyin on an internal definition for a circuit. 

# Future plans
This library is still under construction. If you are looking for specific features that are not yet implemented, please make an issue on the Github page.

Our next plans include, but are not limited to:
- Implement the PSGS algorithm for Pauli Polynomial synthesis
- Improve Qiskit transpiler integration
- Circuit-to-PauliExponential parser for generic circuits
- Improve code structure, package structure, and code style

# Continue reading
i.e. references

1. Huang, Q., Winderl, D., Meijer-Van De Griend, A., & Yeung, R. (2024, September). Redefining Lexicographical Ordering: Optimizing Pauli String Decompositions for Quantum Compiling. In 2024 IEEE International Conference on Quantum Computing and Engineering (QCE) (Vol. 1, pp. 885-896). IEEE. [PDF](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10821336)
1. Winderl, D., Huang, Q., de Griend, A. M. V., & Yeung, R. (2023). Architecture-aware synthesis of stabilizer circuits from clifford tableaus. arXiv preprint arXiv:2309.08972. [PDF](https://arxiv.org/pdf/2309.08972)
1. Meijer–van de Griend, A. (2024). Advances in Quantum Compilation in the NISQ Era (Doctoral dissertation, Doctoral dissertation, University of Helsinki). [PDF](http://hdl.handle.net/10138/569455)
1. Meijer-van de Griend, A. (2025). A comparison of quantum compilers using a DAG-based or phase polynomial-based intermediate representation. Journal of Systems and Software, 221, 112224. [PDF](https://www.sciencedirect.com/science/article/pii/S0164121224002681)
1. Meijer-Van De Griend, A. (2024, March). The Quantum Circuit Model is not a Practical Representation of Quantum Software. In 2024 IEEE International Conference on Software Analysis, Evolution and Reengineering-Companion (SANER-C) (pp. 146-148). IEEE. [PDF](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=10621734)
1. Cowtan, A., Dilkes, S., Duncan, R., Simmons, W., & Sivarajah, S. (2019). Phase gadget synthesis for shallow circuits. arXiv preprint arXiv:1906.01734. [PDF](https://arxiv.org/pdf/1906.01734)
1. Perkkola, F., Salmeperä, I., de Griend, A. M. V., Wang, C. C. J., Bennink, R. S., & Nurminen, J. K. (2025). Optimizing State Preparation for Variational Quantum Regression on NISQ Hardware. arXiv preprint arXiv:2505.17713. [PDF](https://arxiv.org/pdf/2505.17713)

